# Enviroment Setup

In [2]:
# !git clone https://github.com/anantgupta129/TorcHood.git
# %cd TorcHood

In [4]:
# from IPython.display import clear_output

# !python setup.py sdist
# !pip install .

# clear_output()
# %cd ..

# Importing Libraries

In [5]:
import warnings
warnings.filterwarnings("ignore")

import os 
import torch

import lightning.pytorch as pl

from torchood.configs import yolo_config
from torchood.models.yolo_module import  YOLOv3LitModule
from torchood.data.pascal_datamodule import PascalVOCDataModule

# DataLoader & Model

In [6]:
dataloader_config = yolo_config.dataloader_config
dataloader_config["batch_size"] = 32
dataloader_config["num_workers"] = 8

datamodule = PascalVOCDataModule(**dataloader_config)


In [8]:
model = YOLOv3LitModule(learning_rate=1e-3, config=yolo_config, class_labels=yolo_config.PASCAL_CLASSES)

# Training

In [9]:
from lightning.pytorch.callbacks import (
    LearningRateMonitor,
    RichProgressBar,
)
from lightning.pytorch.loggers import WandbLogger


callbacks = [
    RichProgressBar(leave=True),
    LearningRateMonitor(logging_interval="step"),
]
logger = WandbLogger(project="YoloV3-Scratch")
trainer = pl.Trainer(callbacks=callbacks, logger=logger, max_epochs=40, 
                    accelerator = "gpu", precision=16)
trainer.fit(model=model, datamodule=datamodule)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anantgupta129. Use `wandb login --relogin` to force relogin


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name    ┃ Type     ┃ Params ┃
┡━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ net     │ YOLOv3   │ 61.6 M │
│ 1 │ loss_fn │ YoloLoss │      0 │
└───┴─────────┴──────────┴────────┘

Trainable params: 61.6 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 61.6 M                                                                                               
Total estimated model params size (MB): 246

Output()

...

Output()

Output()

Output()

Output()

`Trainer.fit` stopped: `max_epochs=40` reached.


# Evaluation

In [10]:
model = trainer.model
datamodule.setup()

In [11]:
from torchood.models.components.yolo.utils import (
    check_class_accuracy, 
    get_evaluation_bboxes, 
    mean_average_precision,
)
model.to("cuda")
train_class_accuracy, train_no_obj_accuracy, train_obj_accuracy = check_class_accuracy(model, datamodule.train_dataloader(), yolo_config.CONF_THRESHOLD)

518it [01:32,  5.62it/s]

Class accuracy is: 76.434334%
No obj accuracy is: 98.046471%
Obj accuracy is: 67.397293%


In [12]:
loader = datamodule.val_dataloader()

val_class_accuracy, val_no_obj_accuracy, val_obj_accuracy = check_class_accuracy(model, loader, yolo_config.CONF_THRESHOLD)

pred_boxes, true_boxes = get_evaluation_bboxes(
    loader,
    model,
    iou_threshold=yolo_config.NMS_IOU_THRESH,
    anchors=yolo_config.ANCHORS,
    threshold=yolo_config.CONF_THRESHOLD,
)

mapval = mean_average_precision(
    pred_boxes,
    true_boxes,
    iou_threshold=yolo_config.MAP_IOU_THRESH,
    box_format="midpoint",
    num_classes=yolo_config.NUM_CLASSES,
)
print("MAP: ", mapval.item())

155it [00:21,  7.16it/s]

Class accuracy is: 88.029930%
No obj accuracy is: 99.012306%
Obj accuracy is: 71.626488%



155it [03:44,  1.45s/it]


MAP:  0.4827963709831238


In [13]:
logger.log_metrics({"MAP": mapval.item()})
cols = ["train_class_accuracy", "train_no_obj_accuracy", "train_obj_accuracy", "val_class_accuracy", "val_no_obj_accuracy", "val_obj_accuracy"]
rows = [train_class_accuracy, train_no_obj_accuracy, train_obj_accuracy, val_class_accuracy, val_no_obj_accuracy, val_obj_accuracy]
logger.log_table(key="metric", columns=cols, data=[rows])